---
Lógica Computacional TP1.Exercício1
======================
##### G01 - Bruno Dias da Gião A96544, João Luis da Cruz Pereira A95375, David Alberto Agra A95726


---

## Exercício 1 - Enunciado


1. Pretende-se construir o horário semanal de aulas de uma turma.
    1. Existe um conjunto de salas $S$ classificadas em “grandes” e “pequenas”.
    2. O tempo do horário está organizado em “slots” de uma hora. O total do tempo disponível é 5 horas de manhã e 5 horas às tarde.
    3. Existe um conjunto $D\,$ de disciplinas.  Cada disciplina tem um atributo $d\in\{1,2\}$ que classifica a duração de cada sessão (um ou dois “slots”) , um atributo $a\in\{2,3\}$ que define o número de sessões semanais e um atributo $\,s\in\{0,1\}\,$ que diz se a sessão necessita de uma sala grande ou não.
     d. Existe um conjunto $P$ de professores. Cada professor tem associado um conjunto  $h$ das disciplinas que está habilitado a lecionar.
     e. O horário está organizado em sessões concorrentes onde cada sessão é definido por uma disciplina desce que salas e professores verifiquem as seguintes restrições.
        1. Para cada disciplina todas as aulas decorrem na mesma sala e com o mesmo professor.
        2. O número total de horas lecionadas por cada professor está num intervalo de $\pm 20\%\,$do número médio de horas lecionadas pela totalidade dos professores.
        3. Nenhuma sala pode ser ocupada simultaneamente por mais do que uma aula e nenhum professor pode lecionar simultaneamente mais do que uma aula.
        4. Em cada disciplina, cada aula é lecionada por um professor habilitado para essa disciplina e ocorre numa sala de tamanho apropriado à disciplina.
        


    Use o package `ortools` para encontrar uma solução que verifique todas as restrições e maximize o número de partes de dia (manhãs ou tardes) que estão livres de qualquer aula.
---

## Exercício 1 - Solução

### Exercício 1 - Testes

Variáveis de testes (correr antes das restrições)

In [217]:
S = 3
D = 3
P = 3

gp = [0, 1, 1] # salas x grandes pequenas
ph = [[0, 1, 0], [1, 1, 0], [0, 0, 1]] # professores x disciplinas
dd = [2, 2, 2] # disciplinas x duração
na = [2, 2, 2] # diciplinas x nº aulas
ts = [0, 1, 0] # diciplinas x tamanho sala

In [218]:
S = 3
D = 4
P = 4

gp = [0, 1, 1] # salas x grandes pequenas
ph = [[0, 1, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0]] # professores x disciplinas
dd = [2, 2, 2, 2] # disciplinas x duração
na = [2, 2, 2, 2] # diciplinas x nº aulas
ts = [0, 1, 1, 0] # diciplinas x tamanho sala

### Exercício 1 - Código

In [219]:
from ortools.linear_solver import pywraplp
from math import log

solver = pywraplp.Solver.CreateSolver('SCIP')

In [220]:
dias = 5
horas_manha = 5
horas_tarde = 5
h = {}

sala_disc = {}
disc_prof = {}

Criação de dicionário onde iremos guardar a ocorrência de aulas num determinado dia, hora, sala, disciplina e professor com recurso a um BoolVar que o solver irá decidir se é 1 ou 0 (Há aula/Não há aula)

In [221]:
for dia in range(dias):
    h[dia] = {}
    for hora in range(horas_manha+horas_tarde):
        h[dia][hora] = {}
        for s in range(S):
            h[dia][hora][s] = {}
            for d in range(D):
                h[dia][hora][s][d] = {}
                for p in range(P):
                    h[dia][hora][s][d][p] = solver.BoolVar("[%i][%i][%i][%i][%i]" % (dia, hora, s, d, p))

Criação de dois dicionários auxiliares que irão guardar qual é a sala de cada disciplina (única) e qual o professor de cada discilina (único)

In [222]:
for s in range(S):
    sala_disc[s] = {}
    for d in range(D):
        sala_disc[s][d] = solver.BoolVar("[%i][%i]" % (s, d))
        
for d in range(D):
    disc_prof[d] = {}
    for p in range(P):
        disc_prof[d][p] = solver.BoolVar("[%i][%i]" % (d, p))

Esta restrição garante que a duração de cada aula está correta, para isso estamos a percorrer todos os parâmetros (dias, horas, salas, disciplinas, professores) e para cada disciplina verificamos qual a duração de aula pretendida (1 ou 2 horas). Verificamos também casos base (1º hora da manhã, última hora da manhã, 1º hora da tarde, última hora da tarde). Fazemos isto pois no caso de a duração da aula ser de duas horas queremos verificar se há uma aula da mesma disciplina na hora anterior ou na hora seguinte. Para isso fazemos o somatório da hora anterior com a hora seguinte. Nos restantes casos só temos que verificar a hora seguinte quando é a 1º hora manhã/tarde ou a hora anterior quando é a última hora manhã/tarde.

In [223]:
# Duração de aula correta
for dia in range(dias):
    for hora in range(horas_manha + horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P):
                    if (dd[d] == 2 and hora != 0 and hora != horas_manha-1 and hora != horas_manha and hora != horas_manha+horas_tarde-1):
                        solver.Add(h[dia][hora][s][d][p] <= solver.Sum(h[dia][h1][s][d][p] for h1 in range(hora-1, hora+2, 2)))
                    elif (dd[d] == 2 and hora == horas_manha-1):
                        solver.Add(h[dia][hora][s][d][p] <= h[dia][hora-1][s][d][p])
                    elif (dd[d] == 2 and hora == horas_manha):
                        solver.Add(h[dia][hora][s][d][p] <= h[dia][hora+1][s][d][p])
                    elif (dd[d] == 2 and hora == 0):
                        solver.Add(h[dia][hora][s][d][p] <= h[dia][hora+1][s][d][p])
                    elif (dd[d] == 2) and hora == horas_manha+horas_tarde-1:
                        solver.Add(h[dia][hora][s][d][p] <= h[dia][hora-1][s][d][p])
                    else:
                        solver.Add(h[dia][hora][s][d][p] == 0)

Nesta situação, "bloqueamos" cada disciplina e de seguida percorremos todos os parâmetros para a mesma e verificamos que o somatório do número de aulas dessa disciplina é igual ao número de aulas a multiplicar pela duração da mesma.

In [224]:
# Número de horas de aulas correto
for d in range(D):        
    solver.Add(solver.Sum(h[dia][hora][s][d][p] for dia in range(dias) 
                                         for hora in range(horas_manha+horas_tarde)
                                         for s in range(S) 
                                         for p in range(P)
                          ) == na[d]*dd[d])

Para confirmar que o professor atribuído pode dar a disciplina fazemos o contrário e encontramos todas as disciplinas que o professor não está habilitado a dar a disciplina e colocamos todas as ocorrências dessa disciplina com esse professor como 0 (Não há aula). Adicionamos esse valor também ao dicionário auxiliar para posteriormente verificarmos que só há um professor por cada disciplina.

In [225]:
# Professor poder dar a disciplina
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P):
                    if ph[p][d] == 0:
                        solver.Add(h[dia][hora][s][d][p] == 0)
                        solver.Add(disc_prof[d][p] == 0)

No caso de verificar que a sala tem o tamanho adequado à disciplina, usamos um processo idêntico ao de verificar que o professor pode dar a disciplina. Percorremos todos os parâmetros e de seguida verificamos se o tamanho da sala necessário para a disciplina é diferente do tamanho da sala. De seguida adicionamos ao dicionário principal esse valor e também ao auxiliar para posteriormente verificarmos que só há uma sala por cada disciplina.

In [226]:
# Sala tem o tamanho certo
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P):
                    if ts[d] != gp[s]:
                        solver.Add(h[dia][hora][s][d][p] == 0)
                        solver.Add(sala_disc[s][d] == 0)

Um professor só pode dar uma aula por hora então vamos confirmar que em cada dia, em cada hora e para cada professor, o número total de horas de aulas é no máximo 1.

In [227]:
# Uma aula por hora por professor
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for p in range(P):
            solver.Add(solver.Sum(h[dia][hora][s][d][p] for d in range(D) for s in range(S)) <= 1)

In [228]:
# Não pode haver aulas da mesma disciplina ao mesmo tempo
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for d in range(D):
            solver.Add(solver.Sum(h[dia][hora][s][d][p] for s in range(S) for p in range(P)) <= 1)
        
# Um professor por disciplina
for d in range(D):
    solver.Add(solver.Sum(disc_prof[d][p] for p in range(P)) == 1)

for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P): 
                    solver.Add(h[dia][hora][s][d][p] <= disc_prof[d][p])
    
    
# A mesma sala para cada disciplina
for d in range(D):
    solver.Add(solver.Sum(sala_disc[s][d] for s in range(S)) == 1)

for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            for d in range(D):
                for p in range(P):
                    solver.Add(h[dia][hora][s][d][p] <= sala_disc[s][d])
                    
# Uma sala não pode estar ocupada por mais que uma disciplina na mesma hora
for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for s in range(S):
            solver.Add(solver.Sum(h[dia][hora][s][d][p] for d in range(D) for p in range(P)) <= 1)
                    
# Número de horas lecionadas de +/-20% 
total = 0
for i in range(D):
    total += na[d]*dd[d]
media = total // P

for p in range(P):
    solver.Add(media*0.8 <= solver.Sum(disc_prof[d][p] * na[d] * dd[d] for d in range(D)) <= media*1.2)

    
# Maximizar partes de dia livre
sum_hora = [[solver.IntVar(0, D, 'sum_hora') for hora in range(horas_manha+horas_tarde)] for dia in range(dias)]
max_hora = [solver.IntVar(0, D, 'max_hora') for d in range(0, 5)]

for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        sum_hora[dia][hora] = solver.Sum(h[dia][hora][s][d][p] for s in range(S) for d in range(D) for p in range(P))

for dia in range(dias):
    for hora in range(horas_manha+horas_tarde):
        for d in range(D):
            if sum_hora[dia][hora] >= max_hora[d]:
                max_hora[d] = sum_hora[dia][hora]

solver.Maximize(solver.Sum(max_hora[d] for d in range(D)))

In [229]:
def print_horario():
    # Create a 5D list to store the schedule
    schedule = [[[[[0 for _ in range(P)] for _ in range(D)] for _ in range(S)] for _ in range(horas_manha + horas_tarde)] for _ in range(dias)]

    # Extract the solution from the solver and populate the schedule
    for dia in range(dias):
        for hora in range(horas_manha + horas_tarde):
            for s in range(S):
                for d in range(D):
                    for p in range(P):
                        value = h[dia][hora][s][d][p].solution_value()
                        if value == 1:
                            schedule[dia][hora][s][d][p] = value

    # Print the schedule in a human-readable format
    for dia in range(dias):
        print(f"Day {dia + 1}:")
        for hora in range(horas_manha + horas_tarde):
            print(f"Hour {hora + 1}:", end=" ")
            for s in range(S):
                for d in range(D):
                    for p in range(P):
                        if h[dia][hora][s][d][p].solution_value() == 1:
                            print(f"Room {s + 1}, Discipline {d + 1}, Professor {p + 1}")
            print("\n")
        print()  # Separate days with an empty line      

In [230]:
r = solver.Solve()

if r == pywraplp.Solver.OPTIMAL:
    print("Solução encontrada")
    print_horario()
elif r == pywraplp.Solver.INFEASIBLE:
    print("Infeasible")
else:
    print("Não foi encontrada solução")

Solução encontrada
Day 1:
Hour 1: 

Hour 2: 

Hour 3: 

Hour 4: Room 1, Discipline 4, Professor 2


Hour 5: Room 1, Discipline 4, Professor 2


Hour 6: 

Hour 7: Room 1, Discipline 1, Professor 3


Hour 8: Room 1, Discipline 1, Professor 3


Hour 9: 

Hour 10: 


Day 2:
Hour 1: 

Hour 2: 

Hour 3: 

Hour 4: 

Hour 5: 

Hour 6: 

Hour 7: 

Hour 8: 

Hour 9: 

Hour 10: 


Day 3:
Hour 1: 

Hour 2: 

Hour 3: 

Hour 4: 

Hour 5: 

Hour 6: 

Hour 7: 

Hour 8: 

Hour 9: 

Hour 10: 


Day 4:
Hour 1: 

Hour 2: 

Hour 3: 

Hour 4: 

Hour 5: 

Hour 6: 

Hour 7: 

Hour 8: 

Hour 9: 

Hour 10: 


Day 5:
Hour 1: Room 1, Discipline 4, Professor 2


Hour 2: Room 1, Discipline 4, Professor 2


Hour 3: 

Hour 4: 

Hour 5: 

Hour 6: 

Hour 7: Room 2, Discipline 3, Professor 4
Room 3, Discipline 2, Professor 1


Hour 8: Room 2, Discipline 3, Professor 4
Room 3, Discipline 2, Professor 1


Hour 9: Room 1, Discipline 1, Professor 3
Room 2, Discipline 3, Professor 4
Room 3, Discipline 2, Professor 1


Hour 1